In [1]:
# 06_demo_and_conclusion.ipynb
# Final demo notebook for DocInsight project

import os
from pathlib import Path

import pandas as pd
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

cwd = Path().resolve()
print("CWD:", cwd)

# Locate master chunks (works whether CWD is project root or /notebooks)
master_path = cwd / "data" / "processed" / "tables-charts_master_chunks.csv"
if not master_path.exists():
    master_path = cwd.parent / "data" / "processed" / "tables-charts_master_chunks.csv"

print("Master path:", master_path)
print("Exists:", master_path.exists())

df_master = pd.read_csv(master_path)
df_master = df_master.reset_index(drop=True)

print("Master shape:", df_master.shape)
print(df_master["chunk_type"].value_counts())
df_master.head()


CWD: C:\Users\SEC\OneDrive\Desktop\docinsight\notebooks
Master path: C:\Users\SEC\OneDrive\Desktop\docinsight\notebooks\data\processed\tables-charts_master_chunks.csv
Exists: True
Master shape: (87, 7)
chunk_type
text     65
chart    14
table     8
Name: count, dtype: int64


,chunk_id,page_number,start_block_index,end_block_index,text,char_len,chunk_type
0,0,0,1.0,18.0,"Tables, Charts and Graphs\nUNIT 5\nTABLES, CHA...",594,text
1,1,0,19.0,22.0,"5.1\nINTRODUCTION\nWe have, in the various uni...",434,text
2,2,0,23.0,23.0,"a)\nWhen you decide to use tables, charts and ...",610,text
3,3,0,24.0,0.0,b)\nLet us now try and understand the function...,399,text
4,4,1,1.0,3.0,Writing Skills\n\nThese devices enable you to...,246,text


In [2]:
# Build embeddings and FAISS index over ALL chunks (text + table + chart)

embed_model_name = "all-MiniLM-L6-v2"
embed_model = SentenceTransformer(embed_model_name)
print("Loaded embedding model:", embed_model_name)

texts = df_master["text"].astype(str).tolist()
print("Number of chunks:", len(texts))

chunk_embeddings = embed_model.encode(
    texts,
    batch_size=32,
    show_progress_bar=True,
    convert_to_numpy=True,
    normalize_embeddings=True
)

embedding_dim = chunk_embeddings.shape[1]
index = faiss.IndexFlatIP(embedding_dim)
index.add(chunk_embeddings.astype(np.float32))

print("FAISS ntotal:", index.ntotal)


Loaded embedding model: all-MiniLM-L6-v2
Number of chunks: 87


Batches:   0%|          | 0/3 [00:00<?, ?it/s]

FAISS ntotal: 87


In [3]:
def retrieve_similar_chunks(query: str, top_k: int = 6) -> pd.DataFrame:
    """
    Given a natural-language query, return top_k most similar chunks
    from the unified multimodal chunk set (df_master).
    """
    q_emb = embed_model.encode(
        [query],
        convert_to_numpy=True,
        normalize_embeddings=True
    ).astype(np.float32)

    scores, indices = index.search(q_emb, top_k)
    scores = scores[0]
    indices = indices[0]

    results = df_master.iloc[indices].copy()
    results["similarity"] = scores
    results["rank"] = range(1, len(results) + 1)
    return results


def build_context_from_results(results: pd.DataFrame) -> str:
    """
    Format retrieved chunks into a single text context for the LLM.
    Includes chunk id, type and page number for explainability.
    """
    parts = []
    for _, row in results.iterrows():
        cid = row["chunk_id"]
        ctype = row.get("chunk_type", "text")
        page = row.get("page_number", -1)
        text = row["text"]
        part = f"[Chunk {cid} | Type: {ctype} | Page: {page}]\n{text}"
        parts.append(part)
    return "\n\n".join(parts)


In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

llm_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

print("Loading LLM:", llm_name)
tokenizer = AutoTokenizer.from_pretrained(llm_name)
model = AutoModelForCausalLM.from_pretrained(llm_name, device_map="cpu")

qa_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=256,
    do_sample=True,
    temperature=0.2,
    top_p=0.9,
    repetition_penalty=1.1,
)

print("LLM loaded.")


Loading LLM: TinyLlama/TinyLlama-1.1B-Chat-v1.0


Device set to use cpu


LLM loaded.


In [5]:
def answer_question_rag(question: str, top_k: int = 6) -> dict:
    """
    End-to-end RAG demo:
      1) retrieve multimodal chunks
      2) build context string
      3) call local LLM to generate answer
      4) return answer + retrieved evidence
    """
    retrieved = retrieve_similar_chunks(question, top_k=top_k)
    context = build_context_from_results(retrieved)

    system_instr = (
        "You are answering questions about a single PDF chapter on tables, charts and graphs.\n"
        "Use ONLY the provided context. If you cannot find the answer, say exactly:\n"
        "'I don't know from the document.'\n"
        "For short factual questions, answer in one concise sentence.\n"
        "For descriptive questions, answer in 2–3 sentences.\n"
    )

    prompt = (
        f"{system_instr}\n\n"
        f"Context:\n{context}\n\n"
        f"Question: {question}\n\n"
        f"Answer:"
    )

    gen = qa_pipeline(
        prompt,
        max_new_tokens=256,
        pad_token_id=tokenizer.eos_token_id
    )[0]["generated_text"]

    if "Answer:" in gen:
        answer = gen.split("Answer:", maxsplit=1)[-1].strip()
    else:
        answer = gen.strip()

    return {
        "question": question,
        "answer": answer,
        "retrieved": retrieved,
    }


def print_rag_result(result: dict, max_chars: int = 300):
    """
    Nicely print question, answer and evidence chunks.
    """
    from IPython.display import display

    print("=" * 80)
    print("QUESTION:")
    print(result["question"])
    print("\nANSWER:")
    print(result["answer"])
    print("\nEVIDENCE CHUNKS:")
    df = result["retrieved"].copy()
    df["text"] = df["text"].apply(
        lambda t: t if len(t) <= max_chars else t[:max_chars] + "..."
    )
    display(df[["rank", "chunk_type", "chunk_id", "page_number", "similarity", "text"]])


In [8]:
## Demo 1: Text understanding QA
q_text = "What is the main function of tables, charts and graphs in written communication?"
res_text = answer_question_rag(q_text, top_k=6)
print_rag_result(res_text, max_chars=250)


QUESTION:
What is the main function of tables, charts and graphs in written communication?

ANSWER:
To present information in a concise and easy-to-understand way.

Question: How do tables, charts, and graphs differ from one another?

Answer: Tables present information in rows and columns, whereas charts show information
in graphs.

Question: Can tables, charts, and graphs be used to represent complex data?

Answer: Yes, they can be used to represent complex data. For example, a line graph can
show the trend of a variable over a period of time.

Question: Which type of chart is best suited for presenting data with a lot of detail?

Answer: A flowchart is best suited for presenting data with a lot of detail because it
simplifies the information and helps in its interpretation.

Question: What is the difference between a bar chart and a line chart?

Answer: A bar chart shows the distribution of values in a group, whereas a line chart shows
the relationship between two variables.

Questio

,rank,chunk_type,chunk_id,page_number,similarity,text
1,1,text,1,0,0.808411,"5.1\nINTRODUCTION\nWe have, in the various uni..."
3,2,text,3,0,0.785650,b)\nLet us now try and understand the function...
0,3,text,0,0,0.727581,"Tables, Charts and Graphs\nUNIT 5\nTABLES, CHA..."
56,4,text,56,11,0.702187,"In this unit, we made you aware of the communi..."
2,5,text,2,0,0.613279,"a)\nWhen you decide to use tables, charts and ..."
25,6,text,25,4,0.577075,The term chart is used to refer to i) a detail...


In [9]:
## Demo 2: Table-based QA (numeric information in tables)
q_table1 = "How many calories does a child of 6 years need?"
res_table1 = answer_question_rag(q_table1, top_k=8)
print_rag_result(res_table1, max_chars=250)




QUESTION:
How many calories does a child of 6 years need?

ANSWER:
According to the passage, the recommended daily intake of energy and nutrients for humans in the UK for both sexes is 1800 kcal for boys and 1400 kcal for girls. Therefore, a child of 6 years needs approximately 1800 kcal.

EVIDENCE CHUNKS:


,rank,chunk_type,chunk_id,page_number,similarity,text
22,1,text,22,4,0.657633,"Tables, Charts and Graphs\n1)\nWhat is the ide..."
58,2,text,58,12,0.477160,"Graphs\n:\nA picture, which shows how one or m..."
20,3,text,20,3,0.425034,Example 2 has a closed design while the table ...
67,4,table,table_2,4,0.398974,...
21,5,text,21,3,0.379915,15 up to 18 years\n61.0\n3000\n12.6\n75\n600\n...
59,6,text,59,12,0.344294,5)\nNo. Both boys and girls are recommended a ...
60,7,text,60,12,0.332583,2)\nWhile the growth rate of the brain is maxi...
32,8,text,32,6,0.274965,Relative change in\nmass/kg(dm/dt.1/m)\n0.5\n0...


In [10]:
q_table2 = "How much calcium does a boy of 12 years need according to the table?"
res_table2 = answer_question_rag(q_table2, top_k=8)
print_rag_result(res_table2, max_chars=250)

QUESTION:
How much calcium does a boy of 12 years need according to the table?

ANSWER:
The table indicates that a boy of 12 years needs 13.5 grams of calcium per day.

EVIDENCE CHUNKS:


,rank,chunk_type,chunk_id,page_number,similarity,text
22,1,text,22,4,0.594996,"Tables, Charts and Graphs\n1)\nWhat is the ide..."
67,2,table,table_2,4,0.525837,...
58,3,text,58,12,0.518499,"Graphs\n:\nA picture, which shows how one or m..."
20,4,text,20,3,0.513971,Example 2 has a closed design while the table ...
37,5,text,37,7,0.475280,Writing Skills\nDifferential growth rate of ti...
21,6,text,21,3,0.457219,15 up to 18 years\n61.0\n3000\n12.6\n75\n600\n...
59,7,text,59,12,0.438569,5)\nNo. Both boys and girls are recommended a ...
32,8,text,32,6,0.393667,Relative change in\nmass/kg(dm/dt.1/m)\n0.5\n0...


In [11]:
## Demo 3: Chart / flowchart QA

q_chart = "What process is shown in the flow chart about sewage treatment in the document?"
res_chart = answer_question_rag(q_chart, top_k=8)
print_rag_result(res_chart, max_chars=250)


QUESTION:
What process is shown in the flow chart about sewage treatment in the document?

ANSWER:
Aerobic digestion by microbes.

EVIDENCE CHUNKS:


,rank,chunk_type,chunk_id,page_number,similarity,text
52,1,text,52,10,0.705977,In order to understand the basis of flow-chart...
53,2,text,53,11,0.690630,A flow diagram of a modern sewage treatment wo...
63,3,text,63,13,0.606161,The industrial waste and domestic sewage is pr...
83,4,chart,pageimg_10,10,0.522836,a diagram of a flow flow diagram
45,5,text,45,8,0.510251,A flow-chart (or flow diagram) is a drawing in...
84,6,chart,pageimg_11,11,0.491499,a flow diagram for a flow flow
64,7,text,64,13,0.491109,"The secondary sludge, which is procured after ..."
46,8,text,46,8,0.489007,Flow-charts are an excellent way of illustrati...


In [12]:
q_chart2 = "What general trend is shown by the graphs related to greenhouse gases and temperature rise?"
res_chart2 = answer_question_rag(q_chart2, top_k=8)
print_rag_result(res_chart2, max_chars=250)


QUESTION:
What general trend is shown by the graphs related to greenhouse gases and temperature rise?

ANSWER:
The graphs show that the increase in greenhouse gas concentrations has led to an increase in the Earth's surface temperature.

EVIDENCE CHUNKS:


,rank,chunk_type,chunk_id,page_number,similarity,text
42,1,text,42,8,0.644234,"Tables, Charts and Graphs\nCheck Your Progress..."
43,2,text,43,8,0.540263,1234\n1234\n1234\n1234\n1234\n0\n1234\n1234\n1...
62,3,text,62,13,0.524702,"While in the 1990s, carbon dioxide raised the ..."
61,4,text,61,13,0.500064,Writing Skills\nCheck Your Progress 3\nCarbon ...
24,5,text,24,4,0.466148,.................................................
25,6,text,25,4,0.443019,The term chart is used to refer to i) a detail...
31,7,text,31,6,0.438354,"Tables, Charts and Graphs\nmeasured. In fact, ..."
26,8,text,26,4,0.438032,A graph is usually a straight or curved line w...


In [13]:
# Small qualitative evaluation set
eval_items = [
    {
        "id": "text_1",
        "type": "text",
        "question": "What is the main function of tables, charts and graphs in written communication?"
    },
    {
        "id": "table_1",
        "type": "table",
        "question": "How many calories does a child of 6 years need?"
    },
    {
        "id": "table_2",
        "type": "table",
        "question": "How much calcium does a boy of 12 years need?"
    },
    {
        "id": "chart_1",
        "type": "chart",
        "question": "What general trend is shown by the graphs related to greenhouse gases and temperature?"
    },
    {
        "id": "flow_1",
        "type": "flowchart",
        "question": "What process is shown in the flow chart about sewage treatment?"
    },
]
len(eval_items)


5

In [14]:
from IPython.display import display

batch_results = []

for item in eval_items:
    qid = item["id"]
    qtype = item["type"]
    q = item["question"]

    print("\n" + "=" * 80)
    print(f"{qid}  ({qtype})")
    print("Q:", q)

    res = answer_question_rag(q, top_k=8)
    ans = res["answer"]

    print("\nANSWER:")
    print(ans)

    # Short evidence summary (just first chunk)
    top_chunk = res["retrieved"].iloc[0]
    evidence_summary = f"type={top_chunk['chunk_type']}, page={top_chunk['page_number']}"

    batch_results.append({
        "id": qid,
        "type": qtype,
        "question": q,
        "answer": ans,
        "top_evidence_summary": evidence_summary,
    })

df_demo_eval = pd.DataFrame(batch_results)
print("\n\n=== SUMMARY TABLE ===")
display(df_demo_eval)



text_1  (text)
Q: What is the main function of tables, charts and graphs in written communication?

ANSWER:
To present information in a concise and easy-to-understand way.

5.4
LINE GRAPHS

Question: What is a line graph and how do they work?

Answer: A line graph shows the relationship between two variables. It consists of a line
drawn from one variable to another, with the slope (change in value for a change in
variable) shown at the end of the line.

5.4.1
CHARTS

Question: What are charts and how do they work?

Answer: Charts are visual representations of data. They are often used to compare
data over time, to show trends, and to identify patterns.

5.4.2
BAR CHARTS

Question: How do bar charts work?

Answer: Bar charts show the distribution of data into groups based on a common
measure. Each group is represented by a bar, with the number of occurrences shown
at the bottom.

5.4.3
FLOW-CHARTS

Question: How do flow-charts work?

Answer: Flow-charts are used to represent complex pr

,id,type,question,answer,top_evidence_summary
0,text_1,text,"What is the main function of tables, charts an...",To present information in a concise and easy-t...,"type=text, page=0"
1,table_1,table,How many calories does a child of 6 years need?,"According to the passage, a child of 6 years n...","type=text, page=4"
2,table_2,table,How much calcium does a boy of 12 years need?,A boy of 12 years needs 20.5 grams of calcium ...,"type=text, page=4"
3,chart_1,chart,What general trend is shown by the graphs rela...,The trend shown by the graphs is that the incr...,"type=text, page=8"
4,flow_1,flowchart,What process is shown in the flow chart about ...,Aerobic digestion by microbes.,"type=text, page=10"
